Create the proteome xml file from Christian TMT data.   
1) Download E.coli K12 proteome (https://www.uniprot.org/proteomes/UP000000625)    

2) Check if all TMT proteins are inside it  

3) If not, add them

In [1]:
import pyproteinsExt.uniprot as up
%load_ext autoreload
%autoreload 2

In [16]:
tmt_data = "../../data/tsv/TMT-donées brutes_Results_20-0609-0618_VegetativeExp_V2_proteins.tsv"
k12_proteome = "../../data/proteomes/uniprot-proteome_UP000000625.xml"
new_proteome = "../../data/proteomes/uniprot-proteome_UP000000625_+TMT_210126.xml"

### Load proteome

In [17]:
uCollXML = up.EntrySet(collectionXML = k12_proteome)

==> <class 'xml.etree.ElementTree.Element'> <class 'xml.etree.ElementTree.ElementTree'> <==
Acknowledged 0 entries (/Users/guillaumelaunay)
Acknowledged 4391 entries ../../data/proteomes/uniprot-proteome_UP000000625.xml


### Load TMT data and add new proteins to proteome

In [13]:
absent_proteins = []
with open(tmt_data) as f:
    header = f.readline()
    for prot_line in f:
        accession = prot_line.split("\t")[2]
        if not uCollXML.has(accession):
            absent_proteins.append(accession)
            new_entry = uCollXML.get(accession, force_fetch = True)
            uCollXML.add_to_proteome(new_entry)

Looking in XML files/dir collection for Q9S4W2
got to fetch Q9S4W2
Add Q9S4W2 in xml proteome
Looking in XML files/dir collection for P0DP90
got to fetch P0DP90
Add P0DP90 in xml proteome


In [16]:
print(f"{len(absent_proteins)} proteins were not in original proteome :")
print("\n".join(absent_proteins))

2 proteins were not in original proteome :
Q9S4W2
P0DP90


### Write new proteome

In [19]:
uCollXML.serialize_proteome(new_proteome)

Serialize proteome with 4393 entries


#### TESTING PWAS API

* PARSING TSV
* Requests PYTHON API
* Requests NESTJS API

In [31]:
import pandas
import numpy as np

valueField='Abundance Ratio: (pTrc-YfjB) / (pTrc)'

df=pandas.read_csv(tmt_data, sep="\t")
df[valueField] = df[valueField].str.replace(',','.')
df = df.astype({valueField: float} )

DELTA_df = df.loc[(df[valueField] > 1.5)]

print(f"According to field \"{valueField}\", current number of significative proteins {len (DELTA_df['Accession'].tolist())}")

According to field "Abundance Ratio: (pTrc-YfjB) / (pTrc)", current number of significative proteins 53


In [32]:
PYTHON_API_ENDPOINT = "http://localhost:2222/compute"
NESTJS_API_ENDPOINT = "http://localhost:3000/api/pwas/ora"

In [35]:
import requests, json

data = {
    "all_accessions"           : df['Accession'].tolist(),
    "significative_accessions" : DELTA_df['Accession'].tolist(),
    "taxid"                    : "83333", 
    "method"                   : "fisher"
}

ans = requests.post(PYTHON_API_ENDPOINT, json = data)


In [36]:
import requests, json

data = {
    "proteinsExp"           : df['Accession'].tolist(),
    "proteinsDelta" : DELTA_df['Accession'].tolist(),
    "taxid"                    : 83333, 
    "method"                   : "fisher"
}

ans = requests.post(NESTJS_API_ENDPOINT, json = data)
#requests.get("http://localhost:3000/api/pwas")
ans

<Response [201]>

In [25]:
ans.json()

{'children': [{'best': 0.41097679695408007,
   'children': [{'bkgFreq': 0.0027013752455795677,
     'maxMemberCount': 22,
     'name': 'leucine biosynthetic process',
     'pvalue': 0.41097679695408007,
     'table': [[5, 1551], [28, 10665]],
     'uniprotID': ['P0A6A6', 'P0AB80', 'P30125', 'P09151', 'P30126']}],
   'name': 'leucine biosynthetic process_group'},
  {'best': 0.41917231526072896,
   'children': [{'bkgFreq': 0.00036836935166994106,
     'maxMemberCount': 3,
     'name': 'response to lithium ion',
     'pvalue': 0.41917231526072896,
     'table': [[1, 1555], [3, 10694]],
     'uniprotID': ['P0AFA7']},
    {'bkgFreq': 0.00036836935166994106,
     'maxMemberCount': 3,
     'name': 'intracellular pH reduction',
     'pvalue': 0.41917231526072896,
     'table': [[1, 1555], [3, 10694]],
     'uniprotID': ['P0AFA7']}],
   'name': 'response to lithium ion_group'}],
 'name': 'root'}